In [109]:
import pandas as pd
from prophet import Prophet

Install necessary packages, namely pandas and prophet if you do not have already

In [ ]:
df = pd.read_csv('GNPDataset.csv', on_bad_lines='skip')

In [ ]:
df.head()

Data is good, but need to drop last two columns and then reformat csv with date in one column and the visitor number in the second column to feed prophet properly

In [ ]:
#taken and modified from https://github.com/fred-lindsey/projects/blob/main/TSA_NPS_visitor_prediction/wrangle.py

# set 'Year' as the index, so that I can stack the dataframe, ie reduce the dimensions so that I can merge the dataframes
df = df.set_index('Year')
# drop 'Textbox4' and 'AnnualTotal' column. This is a yearly % change and yearly total column for the dataset
df = df.drop(columns=['Textbox4','AnnualTotal'])
# now I'm going to stack, meaning move the columns headers under the index, to reduce the dimensionality of the dataframe
df = df.stack(level=0)
# stacking produces a series when I'm done, and it will need to be converted back into a DF
df = df.to_frame()
# now add a title to the DF's only column
df.columns = ['glacier_national_park_visits']
# reset the index to seperate the current multi-index into distinct year and month columns
df = df.reset_index(drop=False)
# create a composite date column
df['date'] = df.Year.astype(str) + "-" + df.level_1
# convert the date column to datetime object
df.date = pd.to_datetime(df.date)
# set the date as the index and sort index
df = df.set_index('date').sort_index()
# drop Year and level_1 columns that are no longer needed
df = df.drop(columns=['Year', 'level_1'])
display(df)

In [ ]:
df.to_csv(r'C:\Users\divergence\OneDrive - DIVERGENCE ONE\Documents\GitHub\2022-07-DSI-WDX\douglas481\ML and Python Capstone\IndexedData.csv', index=True)

In [110]:
df = pd.read_csv('IndexedData.csv')

In [111]:
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])
df['y'] = df['y'].str.replace(r',', '')
df['y'] = pd.to_numeric(df['y'])
display(df)

,ds,y
0,1979-01-01,6357
1,1979-02-01,3480
2,1979-03-01,11790
3,1979-04-01,15000
4,1979-05-01,65923
...,...,...
511,2021-08-01,670628
512,2021-09-01,533388
513,2021-10-01,149564
514,2021-11-01,26300


In [112]:
df.dtypes

ds    datetime64[ns]
y              int64
dtype: object

Data has been cleaned and formatted to be fed to Prophet, so next we have to train the model

In [113]:
m = Prophet(interval_width=0.95, daily_seasonality=True)
model = m.fit(df)

10:15:42 - cmdstanpy - INFO - Chain [1] start processing
10:15:42 - cmdstanpy - INFO - Chain [1] done processing
